# Revolve Tatbletennis ELO ladder
# Author: Harald Lønsethagen
# Started: 28.08.2017

## Importing Libraries

In [32]:
import pandas as pd
import numpy as np
import matplotlib as plot

## Defining usefull functions

In [33]:
def expected(A, B):
    """
    Calculate expected score of A in a match against B
    :param A: Elo rating for player A
    :param B: Elo rating for player B
    """
    return 1 / (1 + 10 ** ((B - A) / 400))


def elo(old, exp, score, k=32):
    """
    Calculate the new Elo rating for a player
    :param old: The previous Elo rating
    :param exp: The expected score for this match
    :param score: The actual score for this match
    :param k: The k-factor for Elo (default: 32)
    """
    return old + k * (score - exp)

## Importing all played matches

In [34]:
matches_df = pd.read_csv("matches.csv")
print("Matches: " + str(matches_df.shape[0]))
matches_df

Matches: 105


,Player 1,Player 2,Winner,timestamp
0,Paul Frivold,Robert Tamang,Robert Tamang,2017-09-08
1,Paul Frivold,Bo Woelfert,Bo Woelfert,2017-09-08
2,Mons,Ole Andreas,Mons,2017-09-09
3,Mons,Eirik B.,Mons,2017-09-09
4,Mons,Eirik B.,Mons,2017-09-09
5,Paul Frivold,Dino,Paul Frivold,2017-09-09
6,Paul Frivold,Mons,Mons,2017-09-09
7,Bo Woelfert,Dino,Bo Woelfert,2017-09-09
8,Jakob,Mons,Mons,2017-09-10
9,Mons,Yohann,Mons,2017-09-10


## Importing all players 

In [35]:
players_df = pd.read_csv("players.csv")
players_df

,first_name,last_name,short_name,joined_date,id
0,Amund,NaN,rt,17-09-08,1
1,Anders,NaN,mbe,17-09-08,2
2,Andreas B.,NaN,NaN,17-09-07,3
3,Andreas H.,NaN,NaN,17-09-06,4
4,Bo Woelfert,NaN,NaN,17-09-06,5
5,Didrik,NaN,NaN,17-09-04,6
6,Dino,NaN,NaN,17-09-03,7
7,Eirik B.,NaN,NaN,17-09-02,8
8,Even,NaN,NaN,17-09-02,9
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10


## ELO-ladder Dataframe containing the ELO-score to each player

In [36]:
elo_df = players_df
elo_df['elo'] = 1000
elo_df.head()

,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,1000
1,Anders,NaN,mbe,17-09-08,2,1000
2,Andreas B.,NaN,NaN,17-09-07,3,1000
3,Andreas H.,NaN,NaN,17-09-06,4,1000
4,Bo Woelfert,NaN,NaN,17-09-06,5,1000


# Function for calculating ELO

In [37]:
matches_df.iloc(1)[0]

Player 1      Paul Frivold
Player 2     Robert Tamang
Winner       Robert Tamang
timestamp       2017-09-08
Name: 0, dtype: object

In [38]:
length = matches_df.shape[0]
print("Number of matches: " + str(length) + "\n")
for i in range(length):
    print(i)
    match = matches_df.iloc[i]
    print(match)
    player_1 = match["Player 1"]
    player_2 = match["Player 2"]
    elo_rating_a = elo_df[elo_df['first_name'] == player_1]['elo']
    elo_rating_b = elo_df[elo_df['first_name'] == player_2]['elo']

    if(elo_rating_a.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_1) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    if(elo_rating_b.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_2) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    elo_rating_a = int(elo_rating_a.values[0])
    elo_rating_b = int(elo_rating_b.values[0])
    
    
    if(match['Winner'] == player_1): 
        winner_a = 1
        winner_b = 0
    else:
        winner_a = 0
        winner_b = 1
    expected_a  = expected(elo_rating_a, elo_rating_b)
    expected_b  = expected(elo_rating_b, elo_rating_a)
    
    new_elo_a = elo(elo_rating_a, expected_a, winner_a, k = 50)
    new_elo_b = elo(elo_rating_b, expected_b, winner_b, k = 50)
    print("New elo a: " + str(new_elo_a))
    print("New elo b: " + str(new_elo_b))
    print("Player 1: '" + player_1 + "'. Old elo: " + str(elo_rating_a) + "'. New elo: " + str(new_elo_a) + ". exp: " + str(expected_a) + " " )
    print("Player 1: '" + player_2 + "'. Old elo: " + str(elo_rating_b) + "'. New elo: " + str(new_elo_b) + ". exp: " + str(expected_b) + " " )
    elo_df = elo_df.set_index('first_name')
    elo_df = elo_df.set_value(player_1, 'elo', new_elo_a + 1)
    elo_df = elo_df.set_value(player_2, 'elo', new_elo_b + 1)
    elo_df = elo_df.reset_index()
    
elo_df.head()

Number of matches: 105

0
Player 1      Paul Frivold
Player 2     Robert Tamang
Winner       Robert Tamang
timestamp       2017-09-08
Name: 0, dtype: object
New elo a: 975.0
New elo b: 1025.0
Player 1: 'Paul Frivold'. Old elo: 1000'. New elo: 975.0. exp: 0.5 
Player 1: 'Robert Tamang'. Old elo: 1000'. New elo: 1025.0. exp: 0.5 
1
Player 1     Paul Frivold
Player 2      Bo Woelfert
Winner        Bo Woelfert
timestamp      2017-09-08
Name: 1, dtype: object
New elo a: 952.7241972363415
New elo b: 1023.2758027636585
Player 1: 'Paul Frivold'. Old elo: 976'. New elo: 952.7241972363415. exp: 0.46551605527316847 
Player 1: 'Bo Woelfert'. Old elo: 1000'. New elo: 1023.2758027636585. exp: 0.5344839447268315 
2
Player 1            Mons
Player 2     Ole Andreas
Winner              Mons
timestamp     2017-09-09
Name: 2, dtype: object
New elo a: 1025.0
New elo b: 975.0
Player 1: 'Mons'. Old elo: 1000'. New elo: 1025.0. exp: 0.5 
Player 1: 'Ole Andreas'. Old elo: 1000'. New elo: 975.0. exp: 0.5 
3
Pl

Player 1: 'Lasse'. Old elo: 984'. New elo: 956.7752713458497. exp: 0.544494573083008 
43
Player 1          Didrik
Player 2     Bo Woelfert
Winner            Didrik
timestamp     2017-09-22
Name: 43, dtype: object
New elo a: 1030.3334421162808
New elo b: 1070.6665578837192
Player 1: 'Didrik'. Old elo: 998'. New elo: 1030.3334421162808. exp: 0.35333115767438533 
Player 1: 'Bo Woelfert'. Old elo: 1103'. New elo: 1070.6665578837192. exp: 0.6466688423256146 
44
Player 1        Øystein
Player 2         Marcus
Winner           Marcus
timestamp    2017-09-23
Name: 44, dtype: object
New elo a: 949.9389046028585
New elo b: 1023.0610953971415
Player 1: 'Øystein'. Old elo: 973'. New elo: 949.9389046028585. exp: 0.46122190794282847 
Player 1: 'Marcus'. Old elo: 1000'. New elo: 1023.0610953971415. exp: 0.5387780920571715 
45
Player 1     Robert Tamang
Player 2           Øystein
Winner       Robert Tamang
timestamp       2017-09-23
Name: 45, dtype: object
New elo a: 1172.4442869656202
New elo b: 938.

New elo b: 1130.60366159613
Player 1: 'Paul Frivold'. Old elo: 892'. New elo: 881.3963384038701. exp: 0.2120732319225979 
Player 1: 'Bo Woelfert'. Old elo: 1120'. New elo: 1130.60366159613. exp: 0.7879267680774021 
75
Player 1     Paul Frivold
Player 2      Bo Woelfert
Winner        Bo Woelfert
timestamp      2017-09-25
Name: 75, dtype: object
New elo a: 872.3712090835639
New elo b: 1140.628790916436
Player 1: 'Paul Frivold'. Old elo: 882'. New elo: 872.3712090835639. exp: 0.19257581832872078 
Player 1: 'Bo Woelfert'. Old elo: 1131'. New elo: 1140.628790916436. exp: 0.8074241816712792 
76
Player 1      Odin A.S.
Player 2     Andreas B.
Winner       Andreas B.
timestamp    2017-09-26
Name: 76, dtype: object
New elo a: 855.6843958210983
New elo b: 1028.3156041789018
Player 1: 'Odin A.S.'. Old elo: 871'. New elo: 855.6843958210983. exp: 0.3063120835780346 
Player 1: 'Andreas B.'. Old elo: 1013'. New elo: 1028.3156041789018. exp: 0.6936879164219654 
77
Player 1     Sondre N.A.
Player 2    

,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,988
1,Anders,NaN,mbe,17-09-08,2,1061
2,Andreas B.,NaN,NaN,17-09-07,3,1075
3,Andreas H.,NaN,NaN,17-09-06,4,1026
4,Bo Woelfert,NaN,NaN,17-09-06,5,1089


In [39]:
elo_df

,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,988
1,Anders,NaN,mbe,17-09-08,2,1061
2,Andreas B.,NaN,NaN,17-09-07,3,1075
3,Andreas H.,NaN,NaN,17-09-06,4,1026
4,Bo Woelfert,NaN,NaN,17-09-06,5,1089
5,Didrik,NaN,NaN,17-09-04,6,1016
6,Dino,NaN,NaN,17-09-03,7,955
7,Eirik B.,NaN,NaN,17-09-02,8,958
8,Even,NaN,NaN,17-09-02,9,967
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10,998


In [40]:
temp_df = elo_df.sort_values('elo', ascending=False)[['first_name','elo']]
temp_df

,first_name,elo
24,Robert Tamang,1266
19,Mons,1219
4,Bo Woelfert,1089
2,Andreas B.,1075
1,Anders,1061
35,Harald Lønsethagen,1046
31,Henrik,1036
39,Magnus R.B.,1028
38,Sisse,1026
3,Andreas H.,1026


# Save ELO-leaderboard to a csv file

In [41]:
temp_df.to_csv("elo_leaderboard.csv")

# Notes

In [54]:
elo_df = elo_df.reset_index()

In [55]:
elo_df[elo_df['first_name'] == "Harald"]['elo']

0    1001
Name: elo, dtype: int64

In [56]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 3902)
elo_df.reset_index()

,first_name,index,last_name,short_name,joined_date,id,elo
0,Harald,0,Lønsethagen,hlø,17-09-05,1,3902
1,Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [57]:
elo_df = elo_df.reset_index()

In [58]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 900)

,index,last_name,short_name,joined_date,id,elo
first_name,,,,,,
Harald,0,Lønsethagen,hlø,17-09-05,1,900
Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [59]:
exp  = expected(1613, 1609)
exp += expected(1613, 1477)
exp

1.1920564660944573

In [23]:
elo(1613, exp, 2, k=32)  # 1601

1638.8541930849774

In [24]:
a = expected(1613, 1609)
temp_elo = elo(1613, a, 1, k=32)
print("temp_elo: " + str(temp_elo))
a = expected(temp_elo, 1477)
temp_elo = elo(temp_elo, a , 1, k=32)
print("temp_elo after second match: " + str(temp_elo))

temp_elo: 1628.8158013308434
temp_elo after second match: 1638.2378451931402


In [ ]:
# Defining new elo_table with 

# ===============
#          Notes 
# ===============

## Defining usefull DataFrames with Pandas
- All played matches

In [9]:
#matches_df = pd.dataFrame()
item_1 = pd.Series({'timestamp': '00-00-00',
                        'Player 1': 'Harald',
                        'Player 2': 'Petter',
                        'Winner': 'Harald'})
matches_df = pd.DataFrame([item_1])
matches_df.to_csv("matches.csv")
matches_df

,Player 1,Player 2,Winner,timestamp
0,Harald,Petter,Harald,00-00-00
